<a href="https://colab.research.google.com/github/chloewolo/CoCa/blob/main/adp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [46]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/adp_hack/employee_messages.csv', 'r') as f:
  df = pd.read_csv(f)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,record_id,sender_role,source,timestamp,content,emotion,subtlety,is_discrimination,discrimination_type,severity
0,84,employee,chat,2024-03-01 10:05:00,We need to stop hiring people who don’t share ...,anger;disgust,overt,yes,racist,low
1,54,employee,email,2024-03-09 09:30:00,"I feel so invisible in meetings, like my ideas...",sadness,subtle,none,NaN,NaN
2,71,employee,internal_memo,2024-03-01 09:00:00,I don’t see the point in hiring women for tech...,anger;contempt,overt,yes,sexist,high
3,46,manager,email,2024-03-01 08:05:40,I think your way of dealing with things is a b...,condescension,subtle,none,NaN,medium
4,45,employee,text,2024-02-28 14:15:30,The constant changes are giving me anxiety. It...,anxiety,subtle,none,NaN,low


In [47]:
#lowercase messages
df['content'] = df['content'].str.lower()

df['is_discrimination'] = df['is_discrimination'].fillna('no')
df['is_discrimination'] = df['is_discrimination'].replace('none', 'no')
df.loc[df['is_discrimination'] == 'no', 'discrimination_type'] = 'none'
df.loc[df['is_discrimination'] == 'no', 'severity'] = 'n/a'

#remove special characters
df['content'] = df['content'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

#remove extra spaces
df['content'] = df['content'].apply(lambda x: ' '.join(x.split()))

#remove stop words
stop_words = ['and', 'the', 'is', 'in', 'to', 'for', 'on', 'of', 'a', 'an', 'with', 'at', 'by', 'as', 'from']
df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

#remove any missing rows
df.dropna(subset = ['content'], inplace = True)

#remove any duplicates
df.drop_duplicates(subset = ['content'], inplace = True)

#create mew csv file with cleaned data
df.to_csv('cleaned_messages.csv', index = False)
df['is_discrimination'].unique()

array(['yes', 'no'], dtype=object)

In [48]:
df_clean = pd.read_csv('cleaned_messages.csv')
df_clean.head()

,record_id,sender_role,source,timestamp,content,emotion,subtlety,is_discrimination,discrimination_type,severity
0,84,employee,chat,2024-03-01 10:05:00,we need stop hiring people who dont share our ...,anger;disgust,overt,yes,racist,low
1,54,employee,email,2024-03-09 09:30:00,i feel so invisible meetings like my ideas jus...,sadness,subtle,no,none,NaN
2,71,employee,internal_memo,2024-03-01 09:00:00,i dont see point hiring women technical roles ...,anger;contempt,overt,yes,sexist,high
3,46,manager,email,2024-03-01 08:05:40,i think your way dealing things bit unconventi...,condescension,subtle,no,none,NaN
4,45,employee,text,2024-02-28 14:15:30,constant changes are giving me anxiety it feel...,anxiety,subtle,no,none,NaN


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

#initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

#apply TF-IDF to content column (transforms the text into numerical features)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_clean['content'])

#convert TF-IDF matrix to dataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

tfidf_df.to_csv("tfidf_features.csv", index=False)
tfidf_df.head()

,abilities,about,accent,acceptable,adjust,administrative,affairs,affect,affecting,aggressive,...,world,worse,worth,would,years,yet,you,younger,your,youre
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.184804,0.0,0.181255,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


X = tfidf_vectorizer.fit_transform(df['content'])

#the target variable (discrimination or not discrimination)
y = df['is_discrimination'].map({'yes': 1, 'no': 0})  #mapping 'yes' to 1 and 'no' to 0

#split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y

,is_discrimination
0,1
1,0
2,1
3,0
4,0
...,...
95,0
96,1
97,1
98,1


In [51]:
#initialize logistic regression model
lr_model = LogisticRegression(max_iter=1000)

#train the model
lr_model.fit(X_train, y_train)

#predict on the test set
y_pred = lr_model.predict(X_test)

#evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

#save the model using joblib

#import joblib
#joblib.dump(logreg_model, 'logreg_model.pkl')

Accuracy: 0.8500
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.82      0.86        11
           1       0.80      0.89      0.84         9

    accuracy                           0.85        20
   macro avg       0.85      0.85      0.85        20
weighted avg       0.86      0.85      0.85        20



In [52]:
from sklearn.svm import SVC

#initialize SVM model (support vector classifier)
svm_model = SVC(kernel='linear')  # can change kernel to 'rbf' for non-linear

#train the model
svm_model.fit(X_train, y_train)

#predict on the test set
y_pred = svm_model.predict(X_test)

#evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

#save the model using joblib
#import joblib
#joblib.dump(svm_model, 'svm_model.pkl')

Accuracy: 0.8000
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.73      0.80        11
           1       0.73      0.89      0.80         9

    accuracy                           0.80        20
   macro avg       0.81      0.81      0.80        20
weighted avg       0.82      0.80      0.80        20



In [53]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["content"].tolist(),
    df[["is_discrimination", "discrimination_type"]].values.tolist(),
    test_size=0.2, random_state=42
)


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

#tokenizing text
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [59]:
import torch

class DiscriminationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #convert labels to numerical format before creating the tensor
        is_discriminatory = 1 if self.labels[idx][0] == 'yes' else 0

        #map types
        type_mapping = {'sexist': 0, 'racist': 1, 'none': 2}
        #severity_mapping = {'low': 0, 'medium': 1, 'high': 2}

        discrimination_type = type_mapping.get(self.labels[idx][1], 0) #default to 0 if not found
        #severity = severity_mapping.get(self.labels[idx][2], 0) #default to 0 if not found

        item["labels"] = torch.tensor([is_discriminatory, discrimination_type])
        return item



#creating dataset objects
train_dataset = DiscriminationDataset(train_encodings, train_labels)
val_dataset = DiscriminationDataset(val_encodings, val_labels)

In [60]:
from transformers import BertModel
import torch.nn as nn

class MultiOutputBERT(nn.Module):
    def __init__(self):
        super(MultiOutputBERT, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        #output layers
        self.is_discriminatory = nn.Linear(768, 2)  #binary classification (yes/no)
        self.discrimination_type = nn.Linear(768, 3)  #multi (sexist/racist/none)
        #self.severity = nn.Linear(768, num_severities)  #severity level classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        return {
            "is_discriminatory": self.is_discriminatory(pooled_output),
            "discrimination_type": self.discrimination_type(pooled_output),
            #"severity": self.severity(pooled_output),
        }

#num_severities = len(df["severity"].unique())

model = MultiOutputBERT()

In [61]:
import torch.optim as optim

criterion_binary = nn.CrossEntropyLoss()
criterion_multi = nn.CrossEntropyLoss()
#criterion_severity = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [62]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        #compute loss for each output
        loss_discriminatory = criterion_binary(outputs["is_discriminatory"], labels[:, 0])
        loss_type = criterion_multi(outputs["discrimination_type"], labels[:, 1])
        #loss_severity = criterion_severity(outputs["severity"], labels[:, 2])

        loss = loss_discriminatory + loss_type #+ loss_severity  #combined loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 17.3426
Epoch 2, Loss: 11.6465
Epoch 3, Loss: 6.5771


In [63]:
from sklearn.metrics import classification_report

model.eval()
all_preds = {
    "is_discriminatory": [],
    "discrimination_type": [],
}
all_labels = {
    "is_discriminatory": [],
    "discrimination_type": [],

}

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask)

        preds_discriminatory = torch.argmax(outputs["is_discriminatory"], dim=1)
        preds_type = torch.argmax(outputs["discrimination_type"], dim=1)
        #preds_severity = torch.argmax(outputs["severity"], dim=1)

        all_preds["is_discriminatory"].extend(preds_discriminatory.cpu().numpy())
        all_preds["discrimination_type"].extend(preds_type.cpu().numpy())
        #all_preds["severity"].extend(preds_severity.cpu().numpy())

        all_labels["is_discriminatory"].extend(labels[:, 0].cpu().numpy())
        all_labels["discrimination_type"].extend(labels[:, 1].cpu().numpy())
        #all_labels["severity"].extend(labels[:, 2].cpu().numpy())

# Generate classification report for each target separately
for target_name in ["is_discriminatory", "discrimination_type"]:
    print(f"Classification Report for {target_name}:")
    print(classification_report(all_labels[target_name], all_preds[target_name]))

torch.save(model.state_dict(), '/content/drive/MyDrive/adp_hack/discriminatory_model_v1.pth')


Classification Report for is_discriminatory:
              precision    recall  f1-score   support

           0       0.88      0.64      0.74        11
           1       0.67      0.89      0.76         9

    accuracy                           0.75        20
   macro avg       0.77      0.76      0.75        20
weighted avg       0.78      0.75      0.75        20

Classification Report for discrimination_type:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.50      0.80      0.62         5
           2       0.88      0.64      0.74        11

    accuracy                           0.55        20
   macro avg       0.46      0.48      0.45        20
weighted avg       0.61      0.55      0.56        20



In [65]:

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader

#load the saved model
class MultiOutputBERT(nn.Module):
    def __init__(self):
        super(MultiOutputBERT, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.is_discriminatory = nn.Linear(768, 2)
        self.discrimination_type = nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return {
            "is_discriminatory": self.is_discriminatory(pooled_output),
            "discrimination_type": self.discrimination_type(pooled_output),
        }

#load the model from a saved file
model = MultiOutputBERT()
model.load_state_dict(torch.load('/content/drive/MyDrive/adp_hack/discriminatory_model_v1.pth')) #path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def predict_discrimination(message):
    inputs = tokenizer(message, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        # Only pass input_ids and attention_mask to the model
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    is_discriminatory_prob = torch.softmax(outputs["is_discriminatory"], dim=1)
    discrimination_type_prob = torch.softmax(outputs["discrimination_type"], dim=1)

    is_discriminatory = "yes" if torch.argmax(is_discriminatory_prob) == 1 else "no"
    discrimination_type = "sexist" if torch.argmax(discrimination_type_prob) == 0 else "racist"

    return is_discriminatory, discrimination_type


message = "This is an example message."
is_discrim, discrim_type = predict_discrimination(message)
print(f"Is discriminatory: {is_discrim}")
print(f"Discrimination type: {discrim_type}")

<ipython-input-65-8e75e21a40be>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/adp_hack/discriminatory_model_v1.p

Is discriminatory: no
Discrimination type: racist
